<a href="https://colab.research.google.com/github/FatimaEzzedinee/ML-bachelor-course-labs-sp24/blob/main/08_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning SP 2023/2024

- Prof. Cesare Alippi
- Alvise Dei Rossi ([`alvise.dei.rossi@usi.ch`](mailto:alvise.dei.rossi@usi.ch))<br>
- Fatima Ezzeddine ([`fatima.ezzeddine@usi.ch`](mailto:fatima.ezzeddine@usi.ch))<br>
- Alessandro Manenti ([`alessandro.manenti@usi.ch`](mailto:alessandro.manenti@usi.ch))

---
# Lab 08: Forecasting


## Forecasting sunspots

![](https://upload.wikimedia.org/wikipedia/commons/4/47/Solar_Archipelago_-_Flickr_-_NASA_Goddard_Photo_and_Video.jpg)

As a case study we will cosider the problem of forecasting the number of **sunspots** using a dataset collected by the solar physics research department of the Royal Observatory of Belgium ([link](http://sidc.oma.be/)).

According to Wikipedia:

> Sunspots are temporary phenomena on the Sun's photosphere that appear as spots darker than the surrounding areas. They are regions of reduced surface temperature caused by concentrations of magnetic field flux that inhibit convection. Sunspots usually appear in pairs of opposite magnetic polarity. Their number varies according to the approximately 11-year solar cycle.

The datset is available on Kaggle ([link](https://www.kaggle.com/robervalt/sunspots)) and frequently updated.

For semplicity (and reproducibility), we loaded a snapshot of the dataset in the repo of the course.

We start by installing the last version of statsmodels.

In [ ]:
%pip install statsmodels -U

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

url = "https://raw.githubusercontent.com/FatimaEzzedinee/ML-bachelor-course-labs-sp24/main/sunspots.csv"

df = pd.read_csv(url,
                 parse_dates=True,
                 index_col='Date',
                 usecols=['Date', 'Monthly Mean Total Sunspot Number'])
df = df.rename(columns={'Monthly Mean Total Sunspot Number':'sunspots'})

In [ ]:
df.head()

__Note__: the time series we're considering since $d=1$ is said to be _univariate_.

Let's have a look at the data!

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(df.index, df['sunspots'])
plt.xlabel('year')
plt.ylabel('n. sunspots')
plt.grid(alpha=0.25)
plt.show()

First of all let's split our data for model evaluation.

In [ ]:
from sklearn.model_selection import train_test_split

# Prepare the data
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True)

# scale the data
mean = train_df.values.mean()
std = train_df.values.std()

scale = lambda x : (x - mean) / std
inv_scale = lambda x : x * std + mean

train_df = scale(train_df)
test_df = scale(test_df)

# Note that you could do this completely in sklearn with:
# from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
# X_train = scaler.fit_transform(train_df)
# X_test = scaler.transform(test_df)

# THERE IS AN ERROR BEFORE, CAN YOU SPOT IT?

## A simple baseline

Simply use the value at the previous time-step to predict the next one:
$$\hat{y}_{t+1} = x_t$$
This is usually called a Naïve method.

In [ ]:
actuals = inv_scale(test_df.values.ravel())

mse = np.square(actuals[:-1] - actuals[1:]).mean()
print(f"BASELINE MSE: {mse}")
mae = np.abs(actuals[:-1] - actuals[1:]).mean()
print(f"BASELINE MAE: {mae}")

fig, ax1 = plt.subplots(figsize=(20, 5))
ax1.plot(actuals[1:], label='actual')
ax1.plot(actuals[:-1], label='pred')
ax1.set_ylabel('n. sunspots',fontsize=14)
ax1.set_xlabel('idx',fontsize=14)
ax1.set_ylim([0,300])
plt.legend(loc="upper right")
plt.grid(alpha=0.25)
ax2 = ax1.twinx()
ax2.set_ylabel('residual', color="darkgreen", fontsize=14)
ax2.plot(actuals[1:] - actuals[:-1], color="darkgreen", alpha=0.25,label="residuals")
ax2.tick_params(axis='y', labelcolor="darkgreen")
ax2.set_ylim([-150,150])
plt.legend(loc="upper left")
fig.tight_layout()

plt.show()

The fit looks almost perfect, but we should look at the residuals to evaluate the fit. The mismatch between predictions and the actual values can also be seen by zooming in on smaller windows:

In [ ]:
from matplotlib import collections  as mc

lines = [[(i, actuals[i-1]), (i, actuals[i])] for i in range(100,125)]
lc = mc.LineCollection(lines, linewidths=0.5, linestyle="--", colors="darkgreen")

fig, ax1 = plt.subplots(figsize=(10, 5))
ax1.plot(range(100,125),actuals[100:125], label='actual')
ax1.plot(range(100,125),actuals[99:124], label='pred')
ax1.plot([],[],label="residuals",linestyle="--",linewidth=0.5) # fake plot for legend
ax1.set_ylabel('n. sunspots',fontsize=12)
ax1.set_xlabel('idx',fontsize=12)
ax1.add_collection(lc)
plt.legend(loc="upper left")
plt.grid(alpha=0.25)
plt.show()

## Linear models

Let's start with ARIMA models. You can find the documentation [here](https://www.statsmodels.org/devel/generated/statsmodels.tsa.arima.model.ARIMA.html).

As a reminder. An autoregressive model of order p (AR(p)) model is a linear model which considers for the prediction the past p realizations:

$$ \hat{y}_t = \phi_1 x_{t-1} + ... + \phi_p x_{t-p} + c = c + \sum_{i=1}^p \phi_i x_{t-i}$$

A moving average model of order q (MA(q)) is a linear model which considers for the prediction the past q forecasting errors, i.e. defining the residual as $\hat{\eta}_t = y_t - \hat{y}_t$, a MA(q) model gives predictions:

$$\hat{y}_t = \theta_1 \hat{\eta}_{t-1} + ... + \theta_q \hat{\eta}_{t-q} + c = c + \sum_{i=1}^q \theta_i \hat{\eta}_{t-i}; \quad \quad \eta_t \sim WN(0, \sigma^2)$$

Note that this is **NOT** a moving average smoothing.

If you include both the autoregressive and moving average components, you get ARMA(p,q) models. To deal with non-stationary time-series, you need to include also differentiation, i.e. $x'(t) = x_t - x_{t-d}$, to get an ARIMA(p, d, q) model.

Let's start with an AR(p) model:

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings('ignore')

#order refers to (p,d,q) , often called also (p,i,q)
model = ARIMA(train_df, order=(3, 0, 0), trend='n')

fit = model.fit()
print(fit.summary())

# appends test_df to avaiable data
fit = fit.append(test_df, refit=False)

actuals = test_df.values.ravel()
preds = fit.predict(start=len(train_df)).values.ravel()

mse = np.square(inv_scale(preds) - inv_scale(actuals)).mean()
print(f"\n\nTEST MSE: {mse}")
mae = np.abs(inv_scale(preds) - inv_scale(actuals)).mean()
print(f"TEST MAE: {mae}")

Now let's try with a MA(q) model:

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings('ignore')

#ARIMA(p,i,q)
model = ARIMA(train_df, order=(0, 0, 2), trend='n')

fit = model.fit()
print(fit.summary())

# appends test_df to avaiable data
fit = fit.append(test_df, refit=False)

actuals = test_df.values.ravel()
preds = fit.predict(start=len(train_df)).values.ravel()

mse = np.square(inv_scale(preds) - inv_scale(actuals)).mean()
print(f"\n\nTEST MSE: {mse}")
mae = np.abs(inv_scale(preds) - inv_scale(actuals)).mean()
print(f"TEST MAE: {mae}")

That was quite bad... what about combining them?

In [ ]:
#ARIMA(p,i,q)
model = ARIMA(train_df, order=(3, 0, 2), trend='n')

fit = model.fit()
print(fit.summary())

# appends test_df to avaiable data
fit = fit.append(test_df, refit=False)

actuals = test_df.values.ravel()
preds = fit.predict(start=len(train_df)).values.ravel()

mse = np.square(inv_scale(preds) - inv_scale(actuals)).mean()
print(f"\n\nTEST MSE: {mse}")
mae = np.abs(inv_scale(preds) - inv_scale(actuals)).mean()
print(f"TEST MAE: {mae}")

Much better.. Do we need also the differentiation?

In [ ]:
#ARIMA(p,i,q)
model = ARIMA(train_df, order=(3, 1, 2), trend='n')

fit = model.fit()
print(fit.summary())

# appends test_df to avaiable data
fit = fit.append(test_df, refit=False)

actuals = test_df.values.ravel()
preds = fit.predict(start=len(train_df)).values.ravel()

mse = np.square(inv_scale(preds) - inv_scale(actuals)).mean()
print(f"\n\nTEST MSE: {mse}")
mae = np.abs(inv_scale(preds) - inv_scale(actuals)).mean()
print(f"TEST MAE: {mae}")

Seems to obtain about the same results (slightly worse), let's see its predictions:

In [ ]:
preds = fit.predict(start=len(train_df), dynamic=False)

fig, ax1 = plt.subplots(figsize=(20, 5))
ax1.plot(inv_scale(test_df), label='actual')
ax1.plot(inv_scale(preds), label='pred')
ax1.set_ylabel('n. sunspots',fontsize=14)
ax1.set_xlabel('year',fontsize=14)
ax1.set_ylim([0,300])
plt.legend(loc="upper right")
plt.grid(alpha=0.25)
ax2 = ax1.twinx()
ax2.set_ylabel('residual', color="darkgreen", fontsize=14)
ax2.plot(inv_scale(actuals) - inv_scale(preds), color="darkgreen", alpha=0.25,label="residuals")
ax2.tick_params(axis='y', labelcolor="darkgreen")
ax2.set_ylim([-150,150])
plt.legend(loc="upper left")
fig.tight_layout()

plt.show()

In [ ]:
start_idx = 100
end_idx = 125

fig, ax1 = plt.subplots(figsize=(10, 5))
ax1.plot(test_df.index[start_idx:end_idx],inv_scale(test_df)[start_idx:end_idx], label='actual')
ax1.plot(test_df.index[start_idx:end_idx],inv_scale(test_df)[start_idx-1:end_idx-1], label='pred naive')
ax1.plot(inv_scale(preds)[start_idx:end_idx], label='pred arima')
ax1.set_ylabel('n. sunspots',fontsize=12)
ax1.set_xlabel('time',fontsize=12)
plt.legend()
plt.grid(alpha=0.25)
plt.show()

for `dynamic` parameter in `predict` method look [here](https://www.statsmodels.org/0.6.1/generated/statsmodels.tsa.arima_model.ARIMA.predict.html#statsmodels.tsa.arima_model.ARIMA.predict)

In [ ]:
# one-step prediction
y_pred = fit.predict(start=len(train_df), dynamic=False) #dynamic

# recursive prediction
n_last = 300
y_new = fit.predict(start=len(train_df) + len(test_df) - n_last, dynamic=True)

plt.figure(figsize=(12, 5))
plt.plot(inv_scale(test_df), label="actual")
plt.plot(inv_scale(y_pred), label="pred")
plt.plot(inv_scale(y_new), label="new")
plt.xlabel('year')
plt.ylabel('n. sunspots')
plt.legend()
plt.show()

Some final remarks on ARIMA(p,i,q) models:



1.   In the end, how do I choose p, i, q?
We can try to check goodness of fit based on metrics like AIC, BIC with several tests. Or you can use [auto.arima](https://github.com/alkaline-ml/pmdarima) packages.
2.   Can we do more? Yes!
*   we can take seasonality into account (SARIMA)
*   we can add exogenous variables (e.g., date, time of the year, some other useful predictor, ...) (ARIMAX)
*   we can combine the two (SARIMAX)








## Nonlinear Autoregressive models

From the theory:

$$ \hat{y}_t = f_\Theta (x_{t-1} + ... + x_{t-p})$$

with $y_t = x_t$ and $f_\Theta$ a nonlinear function.

### Let's divide data into windows

```
            historical data                   | future observations
--------------------------------------------------------------------
x(1) x(2) ... x(t-p) x(t-p+1) ... x(t-1) x(t) | x(t+1)  x(t+2) ...
                    \________________________/| \____/
                       time window            | value to
                                              | be predicted
```

In [ ]:
# Time windows
def get_time_windows(sequence, window_size):
    time = np.arange(sequence.shape[0])
    xseq = []
    yseq = []
    for t in time[:-window_size]:
        xseq.append(sequence[t:t+window_size])
        yseq.append(sequence[t+window_size]) # one-step prediction
    xseq = np.array(xseq)
    yseq = np.array(yseq)
    #train-test split
    return xseq, yseq.ravel()


In [ ]:
# Prepare the data
p = 24

x_train, y_train = get_time_windows(sequence=train_df.values.ravel(), window_size=p)
x_test, y_test = get_time_windows(sequence=test_df.values.ravel(), window_size=p)
print("training data", x_train.shape, y_train.shape)
print("test data", x_test.shape, y_test.shape)

We can use the usual syntax to build a neural network with pytorch and train it to predict the next value in the time-series.

We can formulate it as a regression problem.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

class FFNN(nn.Module):
    def __init__(self, win_size):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(win_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
model = FFNN(p).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

BATCH_SIZE = 16
n_epochs = 50

In [ ]:
trainset = torch.utils.data.TensorDataset(torch.tensor(x_train, dtype=torch.float32),
                                          torch.tensor(y_train, dtype=torch.float32))
testset = torch.utils.data.TensorDataset(torch.tensor(x_test, dtype=torch.float32),
                                         torch.tensor(y_test, dtype=torch.float32))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=False, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

In [ ]:
def train(model, criterion, optimizer, train_loader, n_epochs, device):
    training_losses = []
    validation_losses = []

    train_length = len(trainloader)
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0.

        for x, y in train_loader:           # Access the training data
            optimizer.zero_grad()           # Zero the gradients
            x, y = x.to(device), y.to(device)
            y_pred = model(x)               # Forward passs

            loss = criterion(y_pred, y)
            loss.backward()                 # Compute the gradients
            optimizer.step()                # Update the weights

            train_loss += loss.item()
        training_losses.append(train_loss/train_length) # Save the loss for plotting

    return training_losses

In [ ]:
training_losses = train(model, criterion, optimizer, trainloader, n_epochs, device)
plt.plot(training_losses)
plt.grid(alpha=0.5)
plt.show()

In [ ]:
predictions = []
actual = []

with torch.no_grad():
    for x, y in testloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        for i in range(len(y_pred)):
          predictions.append(y_pred[i].item())
          actual.append(y[i].item())

preds = np.array(predictions)
actual = np.array(actual)

mse = np.square(inv_scale(preds) - inv_scale(actual)).mean()
print(f"TEST MSE: {mse}")
mae = np.abs(inv_scale(preds) - inv_scale(actual)).mean()
print(f"TEST MAE: {mae}")

In [ ]:
y_pred = model(torch.tensor(x_test, dtype=torch.float32, device=device)).cpu().detach().numpy()

plt.figure(figsize=(12, 5))
plt.plot(np.arange(y_test.shape[0]), inv_scale(y_test), label="actual")
plt.plot(np.arange(y_test.shape[0]), inv_scale(y_pred), label="pred")
plt.legend()
plt.show()

It looks like that the NAR model is not very good.

## Recurrent neural networks

Finally let's try out a more advanced architecture: SimpleRNN networks. Documentation [here](https://keras.io/api/layers/recurrent_layers/simple_rnn/)

In [ ]:
class SimpleRNNModel(nn.Module):
    def __init__(self, win_size):
        super(SimpleRNNModel, self).__init__()
        self.reshape = nn.Linear(win_size, win_size)
        self.rnn = nn.RNN(input_size=1, hidden_size=16, batch_first=True)
        self.fc = nn.Linear(16, 1)

    def forward(self, x):
        x = x.view(x.size(0), -1, 1)
        x, _ = self.rnn(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [ ]:
model = SimpleRNNModel(p).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

BATCH_SIZE = 16
n_epochs = 50

In [ ]:
training_losses = train(model, criterion, optimizer, trainloader, n_epochs, device)
plt.plot(training_losses)
plt.grid(alpha=0.5)
plt.show()

In [ ]:
predictions = []
actual = []

with torch.no_grad():
    for x, y in testloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        for i in range(len(y_pred)):
          predictions.append(y_pred[i].item())
          actual.append(y[i].item())

preds = np.array(predictions)
actual = np.array(actual)

mse = np.square(inv_scale(preds) - inv_scale(actual)).mean()
print(f"TEST MSE: {mse}")
mae = np.abs(inv_scale(preds) - inv_scale(actual)).mean()
print(f"TEST MAE: {mae}")

In [ ]:
y_pred = model(torch.tensor(x_test, dtype=torch.float32, device=device)).cpu().detach().numpy()

plt.figure(figsize=(12, 5))
plt.plot(np.arange(y_test.shape[0]), inv_scale(y_test), label="actual")
plt.plot(np.arange(y_test.shape[0]), inv_scale(y_pred), label="pred")
plt.legend()
plt.show()

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, win_size):
        super(LSTMModel, self).__init__()
        self.reshape = nn.Linear(win_size, win_size)
        self.lstm = nn.LSTM(input_size=1, hidden_size=16, batch_first=True)
        self.fc = nn.Linear(16, 1)

    def forward(self, x):
        x = x.view(x.size(0), -1, 1)
        x, (hn, cn) = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [ ]:
# Prepare the data
p = 150

x_train, y_train = get_time_windows(sequence=train_df.values.ravel(), window_size=p)
x_test, y_test = get_time_windows(sequence=test_df.values.ravel(), window_size=p)
print("training data", x_train.shape, y_train.shape)
print("test data", x_test.shape, y_test.shape)

trainset = torch.utils.data.TensorDataset(torch.tensor(x_train, dtype=torch.float32),
                                          torch.tensor(y_train, dtype=torch.float32))
testset = torch.utils.data.TensorDataset(torch.tensor(x_test, dtype=torch.float32),
                                         torch.tensor(y_test, dtype=torch.float32))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=False, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

In [ ]:
model = LSTMModel(p).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

BATCH_SIZE = 16
n_epochs = 25

In [ ]:
training_losses = train(model, criterion, optimizer, trainloader, n_epochs, device)

plt.plot(training_losses)
plt.show()

In [ ]:
predictions = []
actual = []

with torch.no_grad():
    for x, y in testloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        for i in range(len(y_pred)):
          predictions.append(y_pred[i].item())
          actual.append(y[i].item())

preds = np.array(predictions)
actual = np.array(actual)

mse = np.square(inv_scale(preds) - inv_scale(actual)).mean()
print(f"TEST MSE: {mse}")
mae = np.abs(inv_scale(preds) - inv_scale(actual)).mean()
print(f"TEST MAE: {mae}")

In [ ]:
y_pred = model(torch.tensor(x_test, dtype=torch.float32, device=device)).cpu().detach().numpy()

plt.figure(figsize=(12, 5))
plt.plot(np.arange(y_test.shape[0]), inv_scale(y_test), label="actual")
plt.plot(np.arange(y_test.shape[0]), inv_scale(y_pred), label="pred")
plt.legend()
plt.show()

As you can see it's not always trivial to beat linear autoregressive models.

However... many more things left to explore with non-linear models:

* __Tune hyperparameters!__
* Stack multiple recurrent layers;
* try 1D-CNNs;
* try [GRUs](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html);
* try [Seq2Seq](https://github.com/bentrevett/pytorch-seq2seq) models;
* try [more advanced transformer models](https://huggingface.co/docs/transformers/model_doc/time_series_transformer)
* multi-step predictions;



Want to learn more about forecasting? [This](https://otexts.com/fpp3/) is a good resource to start.

Also take a look at [prophet](https://facebook.github.io/prophet/).